In [ ]:
import sys
sys.executable


In [ ]:
import os
for dirname, _, filenames in os.walk('C:/Users/abi27/Downloads/CTR'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install torch

In [ ]:
!pip install -U deepctr-torch
!pip install category-encoders


In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr_torch.inputs import SparseFeat, get_feature_names,DenseFeat
from deepctr_torch.models import DeepFM

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style="darkgrid")
pd.set_option('display.max_columns', 0)
plt.style.use('ggplot')
pd.options.display.float_format = '{:.2f}'.format
import math
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
import random
import gzip
import category_encoders as ce
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
num_records = 40428967
sample_size = 40028967

skip_values = sorted(random.sample(range(1, num_records), num_records - sample_size))
parse_date = lambda val: pd.to_datetime(val, format='%y%m%d%H')
data = pd.read_csv("C:/Users/abi27/Downloads/CTR/train.csv", parse_dates=['hour'], chunksize=500000, date_parser=parse_date, 
                   skiprows=skip_values)

data = next(data)


In [ ]:
data.shape

In [ ]:
datecolumn=['hour']
for col in datecolumn:
    data[col+'month'] = data[col].dt.month
    data[col+'dayofweek'] = data[col].dt.dayofweek
    data[col+'day'] = data[col].dt.day
    data[col+'hour_time'] = data[col].dt.hour
data.head(2)

In [ ]:
col = ['C15', 'C16', 'C19', 'C21']
for col in col:
    percentiles = data[col].quantile(0.98)
    if data[col].quantile(0.98) < 0.5 * data[col].max():
        data[col][data[col] >= percentiles] = percentiles

In [ ]:
label='click'
target = [label]
dense_features = []
sparse_features = []

for col in (list(data.columns)):
    if data[col].dtype == "object":
        sparse_features.append(col)
    else:
        dense_features.append(col)
dense_features.remove(label)

In [ ]:
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )

In [ ]:
drop_unique_ids=[]

for col in (list(data.columns)):
    if len(list(data[col].unique())) == data.shape[0]:
        drop_unique_ids.append(col)
drop_unique_ids

In [ ]:
if(len(datecolumn)>0):
    data.drop(datecolumn, axis=1, inplace=True)

In [ ]:
if(len(drop_unique_ids)>0):
    data.drop(drop_unique_ids, axis=1, inplace=True)

In [ ]:
dense_features.remove('id')
dense_features.remove('hour')
dense_features.remove('C17')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')
plt.figure(figsize=(5, 4))
sns.countplot(x='click', data=data)
plt.title('Click Distribution')
plt.xlabel('Click (1 = Yes, 0 = No)')
plt.ylabel('Count')
plt.show()


In [ ]:
cardinality = {col: data[col].nunique() for col in categorical_features}
cardinality

In [ ]:
# Pie chart for click distribution
click_counts = data['click'].value_counts()
labels = ['Not Clicked (0)', 'Clicked (1)']
colors = ['skyblue', 'salmon']

plt.figure(figsize=(5, 5))
plt.pie(click_counts, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90, explode=[0.05, 0])
plt.title('Click Distribution')
plt.axis('equal')  # Equal aspect ratio ensures a perfect circle.
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=data, x='device_type', hue='click', palette={0: "steelblue", 1: "coral"})

plt.title('Click Count by Device Type')
plt.xlabel('device_type')
plt.ylabel('Count')
plt.legend(title='click', loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="device_model")
fig.show()

In [ ]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4)
                              for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                              for feat in dense_features]

In [ ]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)
feature_names

In [ ]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
train

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

In [ ]:
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-1, device=device)
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )

In [ ]:
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=10, verbose=2,
                        validation_split=0.15) 

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
pred_ans = model.predict(test_model_input, 256)

In [ ]:
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))

In [ ]:
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test Roc accuracy ", round(roc_auc_score(test[target].values,ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

In [ ]:
from deepctr_torch.models import xDeepFM
model = xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-1, device=device)
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )

In [ ]:
history = model.fit(train_model_input, train[target].values, batch_size=128, epochs=10, verbose=2,
                        validation_split=0.15)

In [ ]:
pred_ans = model.predict(test_model_input, 256)
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))

print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test Roc accuracy ", round(roc_auc_score(test[target].values,ypred), 4))

print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

In [ ]:
from deepctr_torch.models import WDL
model = WDL(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-3, device=device)
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=512, epochs=10, verbose=2,
                        validation_split=0.15)

pred_ans = model.predict(test_model_input, 256)

In [ ]:
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))

print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test Roc accuracy ", round(roc_auc_score(test[target].values,ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))

In [ ]:
from deepctr_torch.models import DCN
model = DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-3, device=device)
model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input, train[target].values, batch_size=512, epochs=10, verbose=2,
                        validation_split=0.15)

pred_ans = model.predict(test_model_input, 256)

In [ ]:
ypred=[]
for i in pred_ans:
    ypred.append(round(i[0]))

print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test Roc accuracy ", round(roc_auc_score(test[target].values,ypred), 4))
print("test f1 score", round(f1_score(test[target].values,ypred), 4))
print("test precision score", round(precision_score(test[target].values,ypred), 4))
print("test recall score", round(recall_score(test[target].values,ypred), 4))
print("test accuracy score", round(accuracy_score(test[target].values,ypred), 4))
